In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
from math import sqrt, pi, log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm
import time



In [23]:
class BSMerton(object):
    def __init__(self, args,b=None):
        self.Type = int(args[0])  # 1 for a Call, - 1 for a put
        self.S = float(args[1])  # Underlying asset price
        self.K = float(args[2])  # Option strike K
        self.r = float(args[3])  # Continuous risk fee rate
        self.q = float(args[4])  # Dividend continuous rate
        self.T = float(args[5]) / 365.0  # Compute time to expiry
        self.sigma = float(args[6])  # Underlying volatility
        self.sigmaT = self.sigma * self.T ** 0.5  # sigma*T for reusability
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT
        if b is None:
            self.b = self.r - self.q
        else:
            self.b = b

        [self.Premium] = self.premium()
        [self.Delta] = self.delta()
        [self.Theta] = self.theta()
        [self.Rho] = self.rho()
        [self.CarryRho] = self.carry_rho()
        [self.Vega] = self.vega()
        [self.Gamma] = self.gamma()
        [self.Phi] = self.phi()
        [self.Charm] = self.dDeltadTime()
        [self.Vanna] = self.dDeltadVol()

    def premium(self):
        tmpprem = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1) - \
                               self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return [tmpprem]

    ############################################
    ############ 1st order greeks ##############
    ############################################

    def delta(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [dfq * norm.cdf(self.d1)]
        else:
            return [dfq * (norm.cdf(self.d1) - 1)]

    # Vega for 1% change in vol
    def vega(self):
        return [0.01 * self.S * e ** (-self.q * self.T) * \
                norm.pdf(self.d1) * self.T ** 0.5]

    # Theta for 1 day change
    def theta(self):
        df = e ** -(self.r * self.T)
        dfq = e ** (-self.q * self.T)
        tmptheta = (1.0 / 365.0) \
                   * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
                      self.sigma / (self.T ** 0.5) + \
                      self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
                                   - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
        return [tmptheta]

    def rho(self):
        df = e ** -(self.r * self.T)
        return [self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)]
    
    from numpy import e

    def carry_rho(self):
        T = self.T
        S = self.S
        r = self.r
        q = self.q
        b = self.b
        d1 = self.d1
        d2 = self.d2
        df = e ** (-r * T)
        dfb = e ** ((b - r) * T)

        if self.Type == 1:  # Call
            carry_rho = T * S * dfb * norm.cdf(d1)
        else:  # Put
            carry_rho = -T * S * dfb * norm.cdf(-d1)

        return [carry_rho * 0.01]


    def phi(self):
        return [0.01 * -self.Type * self.T * self.S * \
                e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1)]

    ############################################
    ############ 2nd order greeks ##############
    ############################################

    def gamma(self):
        return [e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT)]

    # Charm for 1 day change
    def dDeltadTime(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + (-self.q) * norm.cdf(self.d1))]
        else:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + self.q * norm.cdf(-self.d1))]

    # Vanna for 1% change in vol
    def dDeltadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

    # Vomma
    def dVegadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

In [18]:
import datetime as dt
from datetime import date
expiration = dt.datetime(2022,4,15)
current_date = dt.datetime(2022,3,13)

delta= expiration - current_date
delta.days

33

datetime.timedelta(days=33)

In [28]:
# For general accuracy purposes I looked up Implied volatility for AAPL back on 3/13/2022, it was 30.37%


AAPLCall = [1,151.03,165,.0425,.0053, 33,.20]
AAPLPut = [-1,151.03,165,.0425,.0053, 33,.2]


AAPL_Call_Delta = BSMerton(AAPLCall).Delta
AAPL_Call_Gamma = BSMerton(AAPLCall).Gamma
AAPL_Call_Theta = BSMerton(AAPLCall).Theta
AAPL_Call_Vega = BSMerton(AAPLCall).Vega
AAPL_Call_Rho = BSMerton(AAPLCall).Rho
AAPL_Call_Rho_Carry = BSMerton(AAPLCall).CarryRho
AAPL_Call_Charm = BSMerton(AAPLCall).Charm
AAPL_Call_Vanna = BSMerton(AAPLCall).Vanna

AAPL_Call_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Call_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Call_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Call_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Call_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Call_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Call_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Call_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Call_Delta_Exposure,AAPL_Call_Delta)
print(AAPL_Call_Gamma_Exposure,AAPL_Call_Gamma)
print(AAPL_Call_Theta_Exposure,AAPL_Call_Theta)
print(AAPL_Call_Vega_Exposure,AAPL_Call_Vega)
print(AAPL_Call_Rho_Exposure,AAPL_Call_Rho)
print(AAPL_Call_Rho_Carry_Exposure,AAPL_Call_Rho_Carry)
print(AAPL_Call_Charm_Exposure,AAPL_Call_Charm)
print(AAPL_Call_Vanna_Exposure,AAPL_Call_Vanna)


AAPL Net Delta Exposure:  0.08297130333914773
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.022264444821010514
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  0.01102593915636819
AAPL Net Carry Rho Exposure:  0.01132953825011723
AAPL Net Charm Exposure:  -0.003603543622813535
AAPL Net Vanna Exposure:  0.011041137391941661


In [29]:
AAPL_Put_Delta = BSMerton(AAPLPut).Delta
AAPL_Put_Gamma = BSMerton(AAPLPut).Gamma
AAPL_Put_Theta = BSMerton(AAPLPut).Theta
AAPL_Put_Vega = BSMerton(AAPLPut).Vega
AAPL_Put_Rho = BSMerton(AAPLPut).Rho
AAPL_Put_Rho_Carry = BSMerton(AAPLPut).CarryRho
AAPL_Put_Charm = BSMerton(AAPLPut).Charm
AAPL_Put_Vanna = BSMerton(AAPLPut).Vanna

AAPL_Put_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Put_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Put_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Put_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Put_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Put_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Put_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Put_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Put_Delta_Exposure,AAPL_Put_Delta)
print(AAPL_Put_Gamma_Exposure,AAPL_Put_Gamma)
print(AAPL_Put_Theta_Exposure,AAPL_Put_Theta)
print(AAPL_Put_Vega_Exposure,AAPL_Put_Vega)
print(AAPL_Put_Rho_Exposure,AAPL_Put_Rho)
print(AAPL_Put_Rho_Carry_Exposure,AAPL_Put_Rho_Carry)
print(AAPL_Put_Charm_Exposure,AAPL_Put_Charm)
print(AAPL_Put_Vanna_Exposure,AAPL_Put_Vanna)


AAPL Net Delta Exposure:  -0.9165496333661425
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.005317784872060155
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  -0.1375800312273579
AAPL Net Carry Rho Exposure:  -0.1251527180054937
AAPL Net Charm Exposure:  -0.0036180572144972013
AAPL Net Vanna Exposure:  0.011041137391941661
